# Introduction
We have previously extracted and cleaned a dataset of high school (HS) data from students who have submitted applications to CU Boulder This data includes the HS name, its CEEB code, as well as some location information (ZIP, City, and State). The product of this notebook will be a table of HS names, CEEB, and NCES codes, as well as some additional location data where available. This "crosswalk" can then be used to build a more complete HS dataset.

# Script
Begin with some imports and preliminaries:

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import sys
import os
import os.path as path
import pickle as p

from urllib.request import urlopen
import json

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

np.random.seed(8675309)

# useful directories
PROJ_DIR = path.abspath('/home/peter/Desktop/ODA/ceeb_nces_crosswalk')
DATA_DIR = path.join(PROJ_DIR,'data')
MTURK_DIR = path.join(PROJ_DIR,'mturk')

# read in the data
hs_df = pd.read_csv(path.join(DATA_DIR,'ucb_apps_hs.csv'),dtype=str)

/home/peter/Desktop/ODA/ceeb_nces_crosswalk/venv/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# Merging with the Davenport Crosswalk
Let's now load the Davenport CEEB-NCES crosswalk ([source](https://ire.uncg.edu/research/NCES_CEEB_Table/)). We'll use this as the base, and drop all records in our dataset which can be found in the Davenport data.  

In [2]:
# read in the CEEB-NCES crosswalk database TEST
davenport_cross = pd.read_excel(path.join(DATA_DIR,'davenport_nces_ceeb_crosswalk.xlsx'),
                               dtype=str)

# Davenport provided multiple columns for the same items, condense those down
davenport_cross.HS_CITY.fillna(davenport_cross.SAS_MATCH_CITY, inplace=True)
davenport_cross.HS_STATE.fillna(davenport_cross.SAS_MATCH_STATE, inplace=True)
davenport_cross.HS_ZIP.fillna(davenport_cross.SAS_MATCH_ZIP, inplace=True)

# I will drop several of the Davenport columns, and rename the ones I keep
davenport_keep_cols = {'HS_CEEB':'HS_CEEB',
                       'NCESSCH':'HS_NCES',
                       'HS_CITY':'HS_CITY',
                       'HS_NAME':'MATCH_NAME',
                       'HS_ZIP':'HS_POSTAL_CD',
                       'HS_STATE':'HS_STATE'}

davenport_cross.rename(columns=davenport_keep_cols,inplace=True)

# some Davenport crosswalk members were missing an NCES or CEEB code, drop those here
ucb_cross = davenport_cross[list(davenport_keep_cols.values())].dropna(subset=['HS_NCES','HS_CEEB'])

# match the names which CU Boulder has for each school to the Davenport name (renamed to `match_name`)
ucb_cross = ucb_cross.merge(hs_df[['HS_CEEB','HS_NAME']],on='HS_CEEB',how='left')

# determine which CEEBs have been matched through the Davenport crosswalk and drop them from `hs_df`
matched_ceebs = set(ucb_cross.HS_CEEB)
hs_df_less_davenport = hs_df.loc[[_ not in matched_ceebs for _ in hs_df.HS_CEEB]]

# Merging in the NCES PSS 
I will now merge the NCES' [Private School Universe Survey](https://nces.ed.gov/surveys/pss/pssdata.asp) (PSS) into the crosswalk.

In [3]:
pss_df = pd.read_csv(path.join(DATA_DIR,'nces_pss1718.csv'),dtype=str) 
# this dataset contains quite a large amount of columns, for now let's take only a small subset 
pss_cols = ['PPIN','PINST','PADDRS','PCITY','PSTABB','PZIP','PL_CIT','PL_ZIP']
pss_df = pss_df[pss_cols]

I'm going to search through the PSS for schools in the HS dataset which aren't already contained in the Davenport crosswalk. First I will search for the HS's ZIP code in the PSS dataset. Then I'm going fuzzy match the HS name whose CEEB code we want to match against those which share its ZIP code. 

In [4]:
# a little utility script to clean up a schoolname and remove irrelevant text
def name_clean(name):
    name = name.lower()
    name = name.replace('high school','') # leaving "high school" in the school name increases apparent match for no reason  
    name = name.replace('school','') # ditto
    name = name.replace('academy','') # ditto
    
    return(name.strip())

# clean and compare two school names
def hs_name_match_score(name1,name2):
    return(fuzz.ratio(name_clean(name1),name_clean(name2)))

# rewrite `len()` to handle `np.nan`s
def myLen(x):
    if type(x)==str:
        return(len(x))
    else:
        return(np.nan)

hs_df_less_davenport = hs_df_less_davenport.assign(HS_POSTAL_CD=[str(_)[:5] for _ in hs_df_less_davenport.HS_POSTAL_CD])
hs_df_less_davenport.replace({'nan':np.nan},inplace=True)

In [5]:
# this may take a few minutes to run  
match_cutoff = 70

pss_matches = []
for i in range(hs_df_less_davenport.shape[0]):
    row = hs_df_less_davenport.iloc[[i]]
    
    zip_match = (pss_df.PZIP==row.HS_POSTAL_CD.item()) | (pss_df.PL_ZIP==row.HS_POSTAL_CD.item()) 
    scores = [hs_name_match_score(_,row.HS_NAME.item()) for _ in pss_df.loc[zip_match].PINST]
    
    try:
        match = pss_df.loc[zip_match].iloc[[np.argmax(scores)]]
    
        if np.max(scores)>match_cutoff:
            ret_row = {'HS_NAME': row.HS_NAME.item(),
                       'HS_CEEB': row.HS_CEEB.item(),
                       'HS_NCES': match.PPIN.item(),
                       'MATCH_NAME': match.PINST.item(),
                       'MATCH_SCORE': np.max(scores)}
                        
            pss_matches.append(ret_row)

        else:
            ret_row = {'HS_NAME': row.HS_NAME.item(),
                       'HS_CEEB': row.HS_CEEB.item(),
                       'HS_NCES': None,
                       'MATCH_NAME': None,
                       'MATCH_SCORE': None}
    except:
        ret_row = {'HS_NAME': row.HS_NAME.item(),
                   'HS_CEEB': row.HS_CEEB.item(),
                   'HS_NCES': None,
                   'MATCH_NAME': None,
                   'MATCH_SCORE': None} 

pss_matches.append(ret_row)

We'll now add the new PSS crosswalk data into `ucb_cross`, and update the outstanding HS records

In [6]:
pss_cross = pd.DataFrame(pss_matches).replace('None',np.nan).dropna()
pss_cross = hs_df_less_davenport.merge(pss_cross[['HS_CEEB','HS_NCES','MATCH_NAME','MATCH_SCORE']],on='HS_CEEB').drop_duplicates() 

ucb_cross = pd.concat([ucb_cross,pss_cross]).drop_duplicates()

matched_ceebs = set(ucb_cross.HS_CEEB)
hs_df_less_davenport_pss = hs_df.loc[[_ not in matched_ceebs for _ in hs_df.HS_CEEB]]

# Merging in the NCES CCD 
Now let's try and use the Urban Institute's NCES Common Core of Data (CCD) directory [API](https://educationdata-stg.urban.org/documentation/index.html) to get the remaining NCES codes. I'll use a similar approach as with the PSS data: first narrowing the field to schools sharing a ZIP code, followed by fuzzy string matching on names. First define some functions to handle the API:

In [7]:
# the Urban Institute API uses a paged format which returns only
# 1000 records at a time, plus a link to the next "page". Therefore I wrote this little utility functiion
# to run through all the "pages" and collect the results, starting at a specified `url` 
def url_get(url):
    with urlopen(url) as f:
        response = json.loads(f.read())

    count = response['count']

    data = []
    data += response['results']

    while len(data)<count:
        nextURL = response['next']

        with urlopen(nextURL) as f:
            reponse = json.loads(f.read())

        data += response['results']

    return(data)

# look up schools within a certain ZIP code in the NCES' "Common Core of Data" (CCD)
# year is also a variable, but I believe that later years are roughly supersets of earlier years
# so it's advised to just use the most recent year for which data is available
def ccd_directory_zip_lookup(year,zipcode):
    url = f"https://educationdata-stg.urban.org/api/v1/schools/ccd/directory/{year}/?zip_mailing={zipcode}"
    data_mailing = url_get(url)

    url = f"https://educationdata-stg.urban.org/api/v1/schools/ccd/directory/{year}/?zip_location={zipcode}"
    data_location = url_get(url)
    
    return(pd.concat([pd.DataFrame(data_mailing),pd.DataFrame(data_location)]))

# Takes a row of the HS dataset, pulls down CCD records from Urban Institute (UI) API with the same ZIP code
# and fuzzy matches the returned names against the row's HS name
def ui_row_match(row,match_cutoff=match_cutoff):  
    cands = ccd_directory_zip_lookup(2018,row.HS_POSTAL_CD.item())
    
    
    if cands.shape[0]>0:
        scores = [hs_name_match_score(_,row.HS_NAME.item()) for _ in cands.school_name]
    
        match = cands.iloc[[np.argmax(scores)]]
    
        if np.max(scores) > match_cutoff:
            ret_row = {'HS_NAME': row.HS_NAME.item(),
                       'HS_CEEB': row.HS_CEEB.item(),
                       'HS_NCES': match.ncessch.item(),
                       'MATCH_NAME': match.school_name.item(),
                       'MATCH_SCORE': np.max(scores)}
        else:
            ret_row = {'HS_NAME': row.HS_NAME.item(),
                       'HS_CEEB': row.HS_CEEB.item(),
                       'HS_NCES': None,
                       'MATCH_NAME': None,
                       'MATCH_SCORE': None}

    else:
        ret_row = {'HS_NAME': row.HS_NAME.item(),
                   'HS_CEEB': row.HS_CEEB.item(),
                   'HS_NCES': None,
                   'MATCH_NAME': None,
                   'MATCH_SCORE': None}  
    return(ret_row)

Now actually perform the API lookups. Because this takes a few hours, pre-saved results are saved and made available with the GitHub repo. The cell will default to the pre-saved results if they are available

In [8]:
# WARNING: UrbanInstitute API appears to no longer filter on ZIP code and this cell will therefore hang if pre-saved results are not available

save_fname = path.join(DATA_DIR,'presaved_ccd_match_df.csv')
if path.isfile(save_fname):
    ccd_colnames = {'hs_name':'HS_NAME',
                    'hs_ceeb':'HS_CEEB',
                    'ccd_name':'MATCH_NAME',
                    'ccd_nces':'HS_NCES',
                    'score':'MATCH_SCORE'}

    ccd_match_df = pd.read_csv(save_fname,dtype={'hs_ceeb':str,'ccd_nces':str}).drop(columns=['Unnamed: 0'],axis=1).rename(columns=ccd_colnames)

else:
    # WARNING: this may take a long time to run, pre-saved results can be loaded in the cell below 
    ccd_matches = []
    print('Unmatched HS records remaining:')
    print(hs_df_less_davenport_pss.shape[0])
    for i in range(hs_df_less_davenport_pss.shape[0]):
        row = hs_df_less_davenport_pss.iloc[[i]]
        if i%50==0:
            with open(path.join(DATA_DIR,'ccd_matches_inprog.p'),'wb') as f:
                p.dump(ccd_matches,f)
            print(i)
        else:
            pass

        if row.HS_POSTAL_CD.item()==None:
            pass

        else:
            try:
                ccd_matches.append(ui_row_match(row))
            except:
                pass
            
    ccd_match_df = pd.DataFrame(ccd_matches).replace('None',np.nan)
    ccd_match_df.to_csv(path.join(DATA_DIR,'ccd_match_df.csv'))

In [9]:
ccd_cross = ccd_match_df.replace('None',np.nan).dropna()
ccd_cross = hs_df_less_davenport_pss.merge(ccd_cross[['HS_CEEB','HS_NCES','MATCH_NAME','MATCH_SCORE']],on='HS_CEEB').drop_duplicates() 

ccd_cross.head()
ucb_cross = pd.concat([ucb_cross,ccd_cross]).drop_duplicates()

matched_ceebs = set(ucb_cross.HS_CEEB)
hs_df_outstanding = hs_df.loc[[_ not in matched_ceebs for _ in hs_df.HS_CEEB]]

# Checking Match Quality
Now let's do some sanity checks of match quality. First I'll overwrite the `MATCH_SCORE` column (which previously only had values for rows come from the PSS or CCD datasets) so that it includes entries for every row with a `HS_NAME` and a `MATCH_NAME` (some entries in the Davenport cross walk didn't have names, so these I'll leave the score as `NaN`).

In [10]:
def name_check_score(n1,n2):
    if pd.isnull(n1) or pd.isnull(n2):
        return(np.nan)
    
    try:
        return(hs_name_match_score(str(n1),str(n2)))
    
    except:
        return(np.nan)

davenport_ceebs = set(davenport_cross.HS_CEEB)
pss_ceebs = set(pss_cross.HS_CEEB)
ccd_ceebs = set(ccd_cross.HS_CEEB)

def src_check(ceeb):
    if ceeb in davenport_ceebs:
        return(0) # source 0 means NCES found in Davenport crosswalk
    
    elif ceeb in pss_ceebs:
        return(1) # source 1 means NCES found in PSS
    
    elif ceeb in ccd_ceebs:
        return(2) # source 2 means NCES found in CCD (the dataset behind the Urban Inst API)
    
    else:
        return(-1) # source -1 means NCES not found yet
    

ucb_cross['MATCH_SOURCE'] = [src_check(_) for _ in ucb_cross.HS_CEEB]
ucb_cross['MATCH_SCORE'] = [name_check_score(n1,n2) for n1,n2 in zip(ucb_cross.HS_NAME, ucb_cross.MATCH_NAME) ]

ceeb_cts = ucb_cross.groupby('HS_CEEB').count()
ceeb_cts['HS_CEEB'] = ceeb_cts.index
ceeb_dupes = ceeb_cts.HS_CEEB.loc[ceeb_cts.HS_NAME>1]
ucb_cross['HS_DUPED'] = [_ in ceeb_dupes for _ in ucb_cross.HS_CEEB]

print('Number of low-scoring matches by match source:')
ucb_cross.groupby('MATCH_SOURCE').MATCH_SCORE.apply(lambda x: sum(x < match_cutoff))

Number of low-scoring matches by match source:


MATCH_SOURCE
0    358
1     14
2     79
Name: MATCH_SCORE, dtype: int64

There do appear to be some rows from the PSS and CCD datasets with match scores below the cut-off threshold I previously assigned. However, closer inspection reveals that many of these all duplicates of another entry with a higher match score. Although the original `hs_df` dataset had been previously de-duplicated, there are still around 60 CEEB codes which appear more than once in the crosswalk. This might be due to typos in school names, errors/inconsitent ZIP codes (eg. mailing vs. location), etc. Additionally, HS names may change over time, but will retain their CEEB code. When I merged on CEEB codes, duplicated codes were matched with names which are possibly different than what was indicated in the original dataset. For now I will just drop these.

In [11]:
drops = ((ucb_cross.MATCH_SCORE<match_cutoff) & (ucb_cross.MATCH_SOURCE!=0))
ucb_cross = ucb_cross.loc[[not _ for _ in drops]]

Now let's look at the low-quality matches coming from the Davenport crosswalk. It appears as though there are considerably more. I'm going to trust Davenport for the data quality here (at least, compared to the high school names entered on to college applications as they tend to be full over abbreviations and typos). As before I'll overwrite the `HS_NAME` with the `MATCH_NAME` column, and then I'll drop all duplications of `HS_CEEB` and `HS_NCES`.

In [12]:
ucb_cross.HS_NAME.loc[ucb_cross.MATCH_SOURCE==0] = ucb_cross.MATCH_NAME.loc[ucb_cross.MATCH_SOURCE==0]
ucb_cross = ucb_cross.drop_duplicates(subset=['HS_CEEB','HS_NCES']) 

# MTurk Completion
We've now used up all of our "nice" datasets which would allow for automated matching. At this point we'll move the rest over to MTurk. The code below collects and  merges the responses:

In [13]:
def resp_clean(x):
    x = str(x)
    if 'NCES School ID: ' in x:
        x = x.split('NCES School ID: ')
        return(x[-1])
    
    elif ' ' in x:
        return('NA')
    
    elif 'grade' in x.lower():
        return('NA')
    
    else:
        return(x)

def resp_agg(x):
    votes = x.value_counts()
    if np.max(votes) > np.sum(votes)/2.:
        return(votes.idxmax())
    
    else:
        nces = np.argmax([len(_) for _ in votes.index])
        return('NA')

results = pd.read_csv(path.join(MTURK_DIR,'mturk_results_v3_graded.csv'))
results = results.loc[results.Approve=='x']
results.replace(' ', np.nan, inplace=True)

results['Answer'] = results['Answer.HS_NCES'] 
results['Answer'].loc[results['Answer.HS_NO_NCES.on']] = 'NA'

resps_full = results[['WorkerId','Answer','Input.HS_NAME','Input.HS_CEEB']].rename(columns={'Input.HS_NAME':'HS_NAME','Input.HS_CEEB':'HS_CEEB'})
resps_full['Answer'] = resps_full.Answer.apply(resp_clean)
resps_pivot = resps_full.pivot_table(index='HS_CEEB',values='Answer',columns='WorkerId', aggfunc = lambda x: x)

max_width = 4
resps = []
for i in range(resps_pivot.shape[0]):
    row = resps_pivot.iloc[i]
    ceeb = resps_pivot.index[i]
    row = list(row.loc[pd.notnull(row)])
    
    while len(row)<max_width:
        row += [np.nan]
    
    resp_row = {f'resp_{i}':resp for i,resp in zip(range(max_width),row)}
    resp_row['HS_CEEB'] = str(ceeb)
    
    resps.append(resp_row)
    
resps = pd.DataFrame(resps) 
resps['HS_NCES'] = [resp_agg(resps.iloc[i,:4]) for i in range(resps.shape[0])] 
name_ceeb_df = resps_full[['HS_CEEB','HS_NAME']].drop_duplicates()
resps = resps.merge(name_ceeb_df, on='HS_CEEB')

resps[['HS_NAME','HS_CEEB','HS_NCES']].to_csv(path.join(DATA_DIR,'mturk_crosswalk.csv'),index=False)

mturk_cross = resps[['HS_NAME','HS_CEEB','HS_NCES']]

/home/peter/Desktop/ODA/ceeb_nces_crosswalk/venv/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [14]:
# merge in the MTurk responses
mturk_cross = mturk_cross.replace('NA',np.nan).dropna(subset=['HS_NCES'])
mturk_cross = hs_df_outstanding.merge(mturk_cross.rename(columns={'HS_NAME':'MATCH_NAME'})[['HS_CEEB','HS_NCES','MATCH_NAME']],on='HS_CEEB').drop_duplicates() 

ucb_cross = pd.concat([ucb_cross,mturk_cross]).drop_duplicates()

matched_ceebs = set(ucb_cross.HS_CEEB)
hs_df_outstanding = hs_df.loc[[_ not in matched_ceebs for _ in hs_df.HS_CEEB]]

hs_df_outstanding.to_csv(path.join(MTURK_DIR,'outstanding_hs.csv'),index=False)

Collect data for second HiT and export

# Exporting Crosswalk

In [15]:
print('Original Davenport crosswalk size:')
print(davenport_cross.shape)
print('Augmented crosswalk size:')
print(ucb_cross.shape)
print('Growth as percent of original size:')
p = ucb_cross.shape[0] - davenport_cross.shape[0]
p /= davenport_cross.shape[0]  
print(str(round(100*p))+'%')

ucb_cross.to_csv(path.join(PROJ_DIR,'oda_nces_ceeb_crosswalk.csv'),index=False)

Original Davenport crosswalk size:
(18767, 25)
Augmented crosswalk size:
(21391, 10)
Growth as percent of original size:
14%
